In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt

#rms = sqrt(mean_squared_error(y_actual, y_predicted))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
df = pd.read_csv(r'C:\EY-data-science-challenge-pack (1)\train2.csv')
df.head()

,Unnamed: 0,id,LOCATION,TIME,GENDER,Population (% of total),EDU_TERTIARY_PC_WKGPOP,CPI_AGRWTH,GDP_USD_CAP,GOV_SPEND_EDU,...,Population ages 65-69 (as % of specified gender population),Population ages 70-74 (as % of specified gender population),Population ages 75-79 (as % of specified gender population),Population ages 80 and older (as % of specified gender population),TER_Enrolment_rate,"GNI per capita, Atlas method (current US$)",Mortality per 1000 adults,Commercial Services Export,Folds,Quality_of_life_measure
0,1,1,AUS,1993,MEN,49.780616,23.508869,1.753653,19929.78137,5.50922,...,3.744820,2.825824,1.831723,1.659637,65.428513,18930,112.781,1.224386e+10,27,0.172055
1,2,2,AUS,1993,WOMEN,50.219384,21.418512,1.753653,19929.78137,5.50922,...,3.997684,3.396483,2.549315,3.191249,64.678619,18930,61.613,1.224386e+10,27,0.238161
2,3,3,AUS,1994,MEN,49.762137,24.407070,1.969635,21134.14664,5.24752,...,3.738131,2.914956,1.854255,1.710258,66.160980,18890,112.340,1.418181e+10,27,0.173419
3,4,4,AUS,1994,WOMEN,50.237863,21.740088,1.969635,21134.14664,5.24752,...,3.942406,3.479801,2.549259,3.289984,66.824181,18890,62.040,1.418181e+10,27,0.239558
4,5,5,AUS,1995,MEN,49.758261,25.825409,4.627767,22098.57288,5.12905,...,3.716121,2.988446,1.905839,1.739907,68.701340,19320,108.991,1.646077e+10,27,0.173718


In [8]:
df.count()

Unnamed: 0                                                                                         1010
id                                                                                                 1010
LOCATION                                                                                           1010
TIME                                                                                               1010
GENDER                                                                                             1010
Population (% of total)                                                                            1010
EDU_TERTIARY_PC_WKGPOP                                                                              928
CPI_AGRWTH                                                                                          928
GDP_USD_CAP                                                                                         928
GOV_SPEND_EDU                                                   

In [16]:
df = df.replace(to_replace ='..', value = '')
df = df.replace(to_replace ='NA', value = '')
df.GENDER = df.GENDER.replace(to_replace ='MEN', value = 0)
df.GENDER = df.GENDER.replace(to_replace ='WOMEN', value = 1)
df = df.fillna(method ='ffill')
df = df.fillna(method ='bfill')

In [11]:
df.dtypes

Unnamed: 0                                                                                           int64
id                                                                                                   int64
LOCATION                                                                                            object
TIME                                                                                                 int64
GENDER                                                                                               int64
Population (% of total)                                                                            float64
EDU_TERTIARY_PC_WKGPOP                                                                             float64
CPI_AGRWTH                                                                                         float64
GDP_USD_CAP                                                                                        float64
GOV_SPEND_EDU                        

### Extra Features

In [4]:
col_to_use =['CPI_AGRWTH', 'Population ages 55-59 (as % of specified gender population)', 'Population ages 40-44 (as % of specified gender population)', 'Population ages 50-54 (as % of specified gender population)', 'Population ages 35-39 (as % of specified gender population)', 'Population ages 45-49 (as % of specified gender population)', 'Population ages 30-34 (as % of specified gender population)', 'TER_Enrolment_rate', 'Population ages 05-09 (as % of specified gender population)', 'Population ages 15-19 (as % of specified gender population)', 'Population ages 60-64 (as % of specified gender population)', 'Population ages 10-14 (as % of specified gender population)', 'Population ages 65-69 (as % of specified gender population)','Population ages 25-29 (as % of specified gender population)', 'Population ages 20-24 (as % of specified gender population)', 'Population ages 00-04 (as % of specified gender population)', 'EDU_TERTIARY_PC_WKGPOP', 'GDP_USD_CAP', 'Population ages 75-79 (as % of specified gender population)', 'Renewable electricity share of total electricity output (%) [4.1_SHARE.RE.IN.ELECTRICITY]', 'Population ages 70-74 (as % of specified gender population)', 'Population ages 80 and older (as % of specified gender population)', 'GOV_SPEND_EDU', '%_Sanitation_Access', 'Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT]', 'GNI per capita, Atlas method (current US$)', 'Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY]', 'Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC]', 'Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION]', 'Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT]', 'Population (% of total)', 'Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]','Mortality per 1000 adults','Commercial Services Export']
features = df[col_to_use]
labels =  df['Quality_of_life_measure']
features.head()

,CPI_AGRWTH,Population ages 55-59 (as % of specified gender population),Population ages 40-44 (as % of specified gender population),Population ages 50-54 (as % of specified gender population),Population ages 35-39 (as % of specified gender population),Population ages 45-49 (as % of specified gender population),Population ages 30-34 (as % of specified gender population),TER_Enrolment_rate,Population ages 05-09 (as % of specified gender population),Population ages 15-19 (as % of specified gender population),...,Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT],"GNI per capita, Atlas method (current US$)",Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY],Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC],Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION],Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT],Population (% of total),Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM],Mortality per 1000 adults,Commercial Services Export
0,1.753653,4.363110,7.496966,5.125739,7.849939,6.676347,8.245478,65.428513,7.526700,7.674855,...,17220.0,18930,7.502966,8.616992,197804.8632,163236.0,49.780616,2295520.952,112.781,1.224386e+10
1,1.753653,4.220368,7.363331,4.874607,7.807832,6.375044,8.166469,64.678619,7.102381,7.259318,...,17220.0,18930,7.502966,8.616992,197804.8632,163236.0,50.219384,2295520.952,61.613,1.224386e+10
2,1.969635,4.417681,7.440257,5.260645,7.896834,6.916593,8.206122,66.160980,7.450865,7.432178,...,17042.0,18890,7.206736,8.259947,193855.2090,167169.0,49.762137,2346930.376,112.340,1.418181e+10
3,1.969635,4.267955,7.352638,5.003437,7.848636,6.627319,8.134269,66.824181,7.025230,7.011528,...,17042.0,18890,7.206736,8.259947,193855.2090,167169.0,50.237863,2346930.376,62.040,1.418181e+10
4,4.627767,4.480351,7.430364,5.448858,7.926613,7.073719,8.131212,68.701340,7.389884,7.274438,...,16631.0,19320,7.076343,8.217806,199700.4005,172805.0,49.758261,2430093.957,108.991,1.646077e+10


### As is Features

In [47]:
col_to_use =['CPI_AGRWTH', 'Population ages 55-59 (as % of specified gender population)', 'Population ages 40-44 (as % of specified gender population)', 'Population ages 50-54 (as % of specified gender population)', 'Population ages 35-39 (as % of specified gender population)', 'Population ages 45-49 (as % of specified gender population)', 'Population ages 30-34 (as % of specified gender population)', 'TER_Enrolment_rate', 'Population ages 05-09 (as % of specified gender population)', 'Population ages 15-19 (as % of specified gender population)', 'Population ages 60-64 (as % of specified gender population)', 'Population ages 10-14 (as % of specified gender population)', 'Population ages 65-69 (as % of specified gender population)','Population ages 25-29 (as % of specified gender population)', 'Population ages 20-24 (as % of specified gender population)', 'Population ages 00-04 (as % of specified gender population)', 'EDU_TERTIARY_PC_WKGPOP', 'GDP_USD_CAP', 'Population ages 75-79 (as % of specified gender population)', 'Renewable electricity share of total electricity output (%) [4.1_SHARE.RE.IN.ELECTRICITY]', 'Population ages 70-74 (as % of specified gender population)', 'Population ages 80 and older (as % of specified gender population)', 'GOV_SPEND_EDU', '%_Sanitation_Access', 'Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT]', 'GNI per capita, Atlas method (current US$)', 'Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY]', 'Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC]', 'Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION]', 'Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT]', 'Population (% of total)', 'Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]']
features = df[col_to_use]
labels =  df['Quality_of_life_measure']
features.head()

,CPI_AGRWTH,Population ages 55-59 (as % of specified gender population),Population ages 40-44 (as % of specified gender population),Population ages 50-54 (as % of specified gender population),Population ages 35-39 (as % of specified gender population),Population ages 45-49 (as % of specified gender population),Population ages 30-34 (as % of specified gender population),TER_Enrolment_rate,Population ages 05-09 (as % of specified gender population),Population ages 15-19 (as % of specified gender population),...,GOV_SPEND_EDU,%_Sanitation_Access,Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT],"GNI per capita, Atlas method (current US$)",Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY],Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC],Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION],Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT],Population (% of total),Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]
0,1.753653,4.363110,7.496966,5.125739,7.849939,6.676347,8.245478,65.428513,7.526700,7.674855,...,5.50922,100.0,17220.0,18930,7.502966,8.616992,197804.8632,163236.0,49.780616,2295520.952
1,1.753653,4.220368,7.363331,4.874607,7.807832,6.375044,8.166469,64.678619,7.102381,7.259318,...,5.50922,100.0,17220.0,18930,7.502966,8.616992,197804.8632,163236.0,50.219384,2295520.952
2,1.969635,4.417681,7.440257,5.260645,7.896834,6.916593,8.206122,66.160980,7.450865,7.432178,...,5.24752,100.0,17042.0,18890,7.206736,8.259947,193855.2090,167169.0,49.762137,2346930.376
3,1.969635,4.267955,7.352638,5.003437,7.848636,6.627319,8.134269,66.824181,7.025230,7.011528,...,5.24752,100.0,17042.0,18890,7.206736,8.259947,193855.2090,167169.0,50.237863,2346930.376
4,4.627767,4.480351,7.430364,5.448858,7.926613,7.073719,8.131212,68.701340,7.389884,7.274438,...,5.12905,100.0,16631.0,19320,7.076343,8.217806,199700.4005,172805.0,49.758261,2430093.957


#### Aggressive Feature Selection

In [17]:
col_to_use =[ 'Mortality per 1000 adults','TER_Enrolment_rate',  'Population ages 65-69 (as % of specified gender population)','Population ages 25-29 (as % of specified gender population)', 'Population ages 20-24 (as % of specified gender population)', 'Population ages 00-04 (as % of specified gender population)', 'EDU_TERTIARY_PC_WKGPOP', 'GDP_USD_CAP', 'Population ages 75-79 (as % of specified gender population)', 'Renewable electricity share of total electricity output (%) [4.1_SHARE.RE.IN.ELECTRICITY]', 'Population ages 70-74 (as % of specified gender population)', 'Population ages 80 and older (as % of specified gender population)', 'GOV_SPEND_EDU', '%_Sanitation_Access', 'GNI per capita, Atlas method (current US$)', 'Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY]', 'Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC]', 'Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION]', 'Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT]', 'Population (% of total)', 'Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]']
features = features[col_to_use]
features.head()

,Mortality per 1000 adults,TER_Enrolment_rate,Population ages 65-69 (as % of specified gender population),Population ages 25-29 (as % of specified gender population),Population ages 20-24 (as % of specified gender population),Population ages 00-04 (as % of specified gender population),EDU_TERTIARY_PC_WKGPOP,GDP_USD_CAP,Population ages 75-79 (as % of specified gender population),Renewable electricity share of total electricity output (%) [4.1_SHARE.RE.IN.ELECTRICITY],...,Population ages 80 and older (as % of specified gender population),GOV_SPEND_EDU,%_Sanitation_Access,"GNI per capita, Atlas method (current US$)",Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY],Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC],Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION],Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT],Population (% of total),Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]
0,112.781,65.428513,3.744820,7.977674,8.114461,7.483406,23.508869,19929.78137,1.831723,10.55,...,1.659637,5.50922,100.0,18930,7.502966,8.616992,197804.8632,163236.0,49.780616,2295520.952
1,61.613,64.678619,3.997684,7.855364,7.834311,7.050943,21.418512,19929.78137,2.549315,10.55,...,3.191249,5.50922,100.0,18930,7.502966,8.616992,197804.8632,163236.0,50.219384,2295520.952
2,112.340,66.160980,3.738131,7.815958,8.106997,7.480330,24.407070,21134.14664,1.854255,10.19,...,1.710258,5.24752,100.0,18890,7.206736,8.259947,193855.2090,167169.0,49.762137,2346930.376
3,62.040,66.824181,3.942406,7.702864,7.811970,7.043296,21.740088,21134.14664,2.549259,10.19,...,3.289984,5.24752,100.0,18890,7.206736,8.259947,193855.2090,167169.0,50.237863,2346930.376
4,108.991,68.701340,3.716121,7.709084,8.023982,7.436000,25.825409,22098.57288,1.905839,9.62,...,1.739907,5.12905,100.0,19320,7.076343,8.217806,199700.4005,172805.0,49.758261,2430093.957


In [14]:
feature_importance = mutual_info_regression(features, labels)

In [15]:
import operator
feature_list = dict(zip(features.columns,feature_importance))
print(sorted(feature_list.items(), key = operator.itemgetter(1)))

[('CPI_AGRWTH', 0.16279076075444276), ('Population ages 55-59 (as % of specified gender population)', 0.21684798723266585), ('Population ages 40-44 (as % of specified gender population)', 0.22501282183425397), ('Population ages 50-54 (as % of specified gender population)', 0.25046111112728964), ('Population ages 35-39 (as % of specified gender population)', 0.26001847041964066), ('Population ages 45-49 (as % of specified gender population)', 0.28920478786060189), ('Population ages 30-34 (as % of specified gender population)', 0.29479519518420316), ('TER_Enrolment_rate', 0.33233864257511447), ('Population ages 05-09 (as % of specified gender population)', 0.33998816165530599), ('Population ages 15-19 (as % of specified gender population)', 0.34014149650488346), ('Population ages 60-64 (as % of specified gender population)', 0.35793259017871915), ('Population ages 10-14 (as % of specified gender population)', 0.366569698196892), ('Population ages 65-69 (as % of specified gender populatio

In [18]:
xtrain , ytrain, xtest, ytest = features[:670] , labels[:670], features[670:] , labels[670:]  #My own split

In [18]:
xtrain , ytrain = features, labels

### Gradient Boosting regressor with col_to_use (non aggressive)

In [19]:
folds = list(df['Folds'])


# Gradient Boosting with LeaveOneCountryOut cv

In [20]:
# should also be tried in the grid search. (I am not doing it to avoid computational time)
scorer = make_scorer(mean_squared_error) # 
#lol = LeaveOneGroupOut(folds)
#lol.get_n_splits(X, y, labels)
gbm = GradientBoostingRegressor(criterion='mse')
#tuned_parameters = [{'n_estimators': [500,1000], 'max_features': ['auto', 'none']}]
tuned_parameters = [{'n_estimators': [50,100,200,500]}]
clf = GridSearchCV(gbm,tuned_parameters,scoring=scorer, cv= LeaveOneGroupOut())
clf.fit(xtrain, ytrain, folds)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (sqrt(mean), std * 2, params))
    print()
# 0.124 with extra features

Best parameters set found on development set:

{'n_estimators': 100}
Grid scores on development set:

0.120 (+/-0.047) for {'n_estimators': 50}

0.120 (+/-0.047) for {'n_estimators': 100}

0.120 (+/-0.046) for {'n_estimators': 200}

0.120 (+/-0.046) for {'n_estimators': 500}



### Random Forest

In [51]:
# should also be tried in the grid search. (I am not doing it to avoid computational time)
scorer = make_scorer(mean_squared_error) # 
forest = RandomForestRegressor(criterion='mse')
#tuned_parameters = [{'n_estimators': [500,1000], 'max_features': ['auto', 'none']}]
tuned_parameters = [{'n_estimators': [200,500,800,1000],'max_features': ['sqrt', 'log2','auto'] , 'max_depth' : [2,1,10,None] }]
clf = GridSearchCV(forest,tuned_parameters,scoring=scorer, cv=LeaveOneGroupOut())
clf.fit(xtrain, ytrain, folds)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (sqrt(mean), std * 2, params))
    print()

Best parameters set found on development set:

{'max_depth': 1, 'max_features': 'auto', 'n_estimators': 1000}
Grid scores on development set:

0.109 (+/-0.049) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 200}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 500}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 800}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 1000}

0.110 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 200}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 500}

0.110 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 800}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 1000}

0.120 (+/-0.050) for {'max_depth': 2, 'max_features': 'auto', 'n_estimators': 200}

0.119 (+/-0.049) for {'max_depth': 2, 'max_features': 'auto', 'n_estimators': 500}

0.119 (+/-0.049

### Random Forest with Zain's Features

In [8]:
# should also be tried in the grid search. (I am not doing it to avoid computational time)
scorer = make_scorer(mean_squared_error) # 
forest = RandomForestRegressor(criterion='mse')
#tuned_parameters = [{'n_estimators': [500,1000], 'max_features': ['auto', 'none']}]
tuned_parameters = [{'n_estimators': [200,500,800,1000],'max_features': ['sqrt', 'log2','auto'] , 'max_depth' : [2,1,10,None] }]
clf = GridSearchCV(forest,tuned_parameters,scoring=scorer, cv=LeaveOneGroupOut())
clf.fit(xtrain, ytrain, folds)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (sqrt(mean), std * 2, params))
    print()

Best parameters set found on development set:

{'max_depth': 1, 'max_features': 'auto', 'n_estimators': 1000}
Grid scores on development set:

0.110 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 200}

0.110 (+/-0.051) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 500}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 800}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 1000}

0.109 (+/-0.049) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 200}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 500}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 800}

0.109 (+/-0.050) for {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 1000}

0.119 (+/-0.049) for {'max_depth': 2, 'max_features': 'auto', 'n_estimators': 200}

0.119 (+/-0.049) for {'max_depth': 2, 'max_features': 'auto', 'n_estimators': 500}

0.119 (+/-0.049

## Agressive Feature Selection RF

In [21]:
# should also be tried in the grid search. (I am not doing it to avoid computational time)
scorer = make_scorer(mean_squared_error) # 
forest = RandomForestRegressor(criterion='mse')
#tuned_parameters = [{'n_estimators': [500,1000], 'max_features': ['auto', 'none']}]
tuned_parameters = [{'n_estimators': [200,500,800,1000],'max_features': ['sqrt', 'log2']  }]
clf = GridSearchCV(forest,tuned_parameters,scoring=scorer, cv=LeaveOneGroupOut())
clf.fit(xtrain, ytrain, folds)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (sqrt(mean), std * 2, params))
    print()

Best parameters set found on development set:

{'max_features': 'sqrt', 'n_estimators': 200}
Grid scores on development set:

0.112 (+/-0.047) for {'max_features': 'sqrt', 'n_estimators': 200}

0.111 (+/-0.047) for {'max_features': 'sqrt', 'n_estimators': 500}

0.110 (+/-0.047) for {'max_features': 'sqrt', 'n_estimators': 800}

0.110 (+/-0.046) for {'max_features': 'sqrt', 'n_estimators': 1000}

0.111 (+/-0.047) for {'max_features': 'log2', 'n_estimators': 200}

0.111 (+/-0.047) for {'max_features': 'log2', 'n_estimators': 500}

0.110 (+/-0.046) for {'max_features': 'log2', 'n_estimators': 800}

0.110 (+/-0.047) for {'max_features': 'log2', 'n_estimators': 1000}



In [22]:
import pickle
filename = 'EY_RF_GroupFolds_withextrafeatures_agg.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [44]:
#feature selection (normal)
ypred = clf.predict(xtest)
rms = sqrt(mean_squared_error(ytest, ypred))
print ('Test Score: %0.3f'%(rms))

Test Score: 0.109


## TEST SET PREDICTION/SUBMISSION

In [23]:
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv(r'C:\EY-data-science-challenge-pack (1)\test2.csv')
df = df.replace(to_replace ='..', value = '')
df.GENDER = df.GENDER.replace(to_replace ='MEN', value = 0)
df.GENDER = df.GENDER.replace(to_replace ='WOMEN', value = 1)
df = df.fillna(method ='ffill')
df = df.fillna(method ='bfill')
df1 = df[col_to_use]

df1['ypred'] = loaded_model.predict(df1)

#df1.to_csv(r'C:\EY-data-science-challenge-pack (1)\submit_EY_RF_GroupFolds_withextrafeatures.csv')



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [24]:
df2 = pd.DataFrame()
df2['id'] = df['id']
df2['target'] = df1['ypred']

df2.to_csv(r'C:\EY-data-science-challenge-pack (1)\submit_EY_RF_GroupFolds_withextrafeatures_agg.csv')
